In [1]:
import torch
import dqc
import time

dtype = torch.float64

In [2]:
atomzs = torch.tensor([8, 1, 1], dtype=torch.int64)
# from CCCBDB (calculated geometry for H2O for HF/cc-pvdz)
atomposs = (torch.tensor([
    [0.0, 0.0, 0.1157],
    [0.0, 0.7488, -0.4629],
    [0.0, -0.7488, -0.4629],
], dtype=dtype) * 1.8897259886).requires_grad_()
efield = torch.zeros(3, dtype=dtype).requires_grad_()
grad_efield = torch.zeros((3, 3), dtype=dtype).requires_grad_()

efields = (efield, grad_efield)
mol = dqc.Mol(moldesc=(atomzs, atomposs), basis="cc-pvdz", dtype=dtype, efield=efields)
qc = dqc.HF(mol).run()

In [3]:
# dipole
dip = dqc.edipole(qc, unit="debye")
print(dip)

tensor([ 3.8799e-16,  3.8770e-14, -2.0442e+00], dtype=torch.float64,
       grad_fn=<MulBackward0>)


In [4]:
# quadrupole
quad = dqc.equadrupole(qc, unit="debye*angst")
print(quad)

tensor([[-7.0084e+00, -1.5913e-16,  1.4936e-16],
        [-1.5913e-16, -4.1405e+00, -1.2222e-14],
        [ 1.4936e-16, -1.2222e-14, -5.8839e+00]], dtype=torch.float64,
       grad_fn=<MulBackward0>)


In [5]:
# ir spectrum
freq, ir_ints = dqc.ir_spectrum(qc, freq_unit="cm^-1", ints_unit="km/mol")
print(freq, ir_ints)

tensor([4.2116e+03, 4.1133e+03, 1.7757e+03, 1.4622e+01, 1.4497e+01, 1.4258e+01,
        2.0408e-02, 4.8197e-03, 3.0302e-03], dtype=torch.float64,
       grad_fn=<MulBackward0>) tensor([6.0469e+01, 2.1172e+01, 8.0686e+01, 1.5744e-15, 1.0209e+02, 2.9461e+02,
        5.0680e-09, 6.2067e-18, 5.4876e-07], dtype=torch.float64,
       grad_fn=<MulBackward0>)


In [7]:
# raman spectrum
freq, raman_ints = dqc.raman_spectrum(qc, freq_unit="cm^-1", ints_unit="angst^4/amu")
print(freq)
print(raman_ints)

tensor([4.2116e+03, 4.1133e+03, 1.7757e+03, 1.4622e+01, 1.4497e+01, 1.4258e+01,
        2.0408e-02, 4.8197e-03, 3.0302e-03], dtype=torch.float64,
       grad_fn=<MulBackward0>)
tensor([3.4789e+01, 6.8881e+01, 4.7901e+00, 5.4320e+00, 7.8389e-01, 2.8784e+00,
        3.8918e-11, 2.0156e-17, 5.3614e-09], dtype=torch.float64,
       grad_fn=<MulBackward0>)
